In [1]:
import numpy as np
import sympy as sym 
import scipy.optimize as sopt
from scipy.integrate import ode
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, HoverTool, CustomJS
output_notebook()

Loading BokehJS ...

In [18]:
delta_1,omega_1,delta_2,omega_2 = sym.symbols('delta_1,omega_1,delta_2,omega_2')
H_1,D_1,H_2,D_2 = sym.symbols('H_1,D_1,H_2,D_2')
p_load_1,p_load_2 = sym.symbols('p_load_1,p_load_2')
E_1_ref,E_2_ref,p_m_1,p_m_2 = sym.symbols('E_1_ref,E_2_ref,p_m_1,p_m_2')
X_tie,Omega_b,V = sym.symbols('X_tie,Omega_b,V') 
x_turbine_1,x_turbine_2 = sym.symbols('x_turbine_1,x_turbine_2') 
T_b_1,T_b_2,T_c_1,T_c_2 = sym.symbols('T_b_1,T_b_2,T_c_1,T_c_2') 
Droop_1,Droop_2 = sym.symbols('Droop_1,Droop_2') 
K_pgov_1, K_igov_1, K_imw_1 = sym.symbols('K_pgov_1,K_igov_1,K_imw_1')
K_pgov_2, K_igov_2, K_imw_2 = sym.symbols('K_pgov_2,K_igov_2,K_imw_2') 
x_lc_1,x_lc_2 = sym.symbols('x_lc_1,x_lc_2') 
gov_ref_1,gov_ref_2 = sym.symbols('gov_ref_1,gov_ref_2') 
x_pid_1,x_pid_2 = sym.symbols('x_pid_1,x_pid_2') 
K_stab_1, K_stab_2   = sym.symbols('K_stab_1, K_stab_2') 

In [19]:
delta_coi = (delta_1 * H_1 + delta_2 * H_2)/(H_1+H_2) 
omega_coi = (omega_1 * H_1 + omega_2 * H_2)/(H_1+H_2) 

E_1 = E_1_ref -K_stab_1*(omega_1 - omega_coi)
E_2 = E_2_ref -K_stab_2*(omega_2 - omega_coi)
p_e_1 = E_1*V/X_tie*sym.sin(delta_1 - delta_coi) + p_load_1    
p_e_2 = E_2*V/X_tie*sym.sin(delta_2 - delta_coi) + p_load_2   

u_pid_1 =  gov_ref_1 - omega_1 + x_lc_1 - Droop_1 * p_e_1  
u_pid_2 =  gov_ref_2 - omega_2 + x_lc_2 - Droop_2 * p_e_2  

p_s_1 = u_pid_1*K_pgov_1 + x_pid_1
p_s_2 = u_pid_2*K_pgov_2 + x_pid_2


p_m_1 = T_c_1/T_b_1*(p_s_1-x_turbine_1)+x_turbine_1
p_m_2 = T_c_2/T_b_2*(p_s_2-x_turbine_2)+x_turbine_2




dx_pid_1 = K_igov_1*u_pid_1
dx_pid_2 = K_igov_2*u_pid_2

ddelta_1 = Omega_b*(omega_1 - omega_coi)
ddelta_2 = Omega_b*(omega_2 - omega_coi)

domega_1 = 1.0/(2*H_1)*(p_m_1 - p_e_1  -  D_1*(omega_1-omega_coi))
domega_2 = 1.0/(2*H_2)*(p_m_2 - p_e_2  -  D_2*(omega_2-omega_coi))

dx_turbine_1 =  1.0/T_b_1*(p_s_1-x_turbine_1)
dx_turbine_2 =  1.0/T_b_2*(p_s_2-x_turbine_2)



In [20]:
f = sym.Matrix([[ddelta_1],[domega_1],[dx_turbine_1],[dx_pid_1],[ddelta_2],[domega_2],[dx_turbine_2],[dx_pid_2]])
x = sym.Matrix([ [delta_1], [omega_1], [x_turbine_1],[ x_pid_1], [delta_2], [omega_2], [x_turbine_2],[x_pid_2]])

F_x = f.jacobian(x)

In [21]:
F_x

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                        0,                                                                                                                                                                                                                                             Omega_b*(-H_1/(H_1 + H_2) + 1),                         0,                     0,                                                                                                                                                                                                                                                                                                                      

In [130]:
p_load_1_val = 0.0
p_load_2_val = 0.0
params_ss = {'H_1':7.0,
          'D_1':5.0,
          'H_2':7.0,
          'D_2':5.0,
          "T_b_1":10.0,
          "T_b_2":0.5,
          "T_c_1":2.0,
          "T_c_2":0.0,             
          "Droop_1":0.05,
          "Droop_2":0.05,
          "K_stab_1":-0.0,
          "K_stab_2":-0.0,
          "Droop_2":0.05,
          'X_tie':1.0,
          'Omega_b':2*np.pi*50.0,
          'V':1.0,
          'E_1_ref':1.0,
          'E_2_ref':1.0,
          'p_load_1':p_load_1_val,
          'p_load_2':p_load_2_val,
          'K_pgov_1':10.0, 
          'K_igov_1':2,
          'K_pgov_2':10.0, 
          'K_igov_2':2,
          'gov_ref_1':1.0,
          'gov_ref_2':1.0, 
          'x_lc_1':0.0,
          'x_lc_2':0.0
         }

u_ss = sym.Matrix([[delta_1],[omega_1],[x_turbine_1],[x_pid_1],[delta_2],[omega_2],[x_turbine_2],[x_pid_2]])
F_x_ss = f.jacobian(u_ss)

f_n   = sym.lambdify((u_ss), f.subs(params_ss),'numpy') # returns a numpy-ready function
jac_ss = sym.lambdify((u_ss), F_x_ss.subs(params_ss),'numpy') # returns a numpy-ready function

In [131]:
f.subs(params_ss)

Matrix([
[                                                                                                                                           157.07963267949*omega_1 - 157.07963267949*omega_2],
[-0.321428571428571*omega_1 + 0.178571428571429*omega_2 + 0.0142857142857143*x_pid_1 + 0.0571428571428571*x_turbine_1 - 0.0785714285714286*sin(0.5*delta_1 - 0.5*delta_2) + 0.142857142857143],
[                                                                                                    -1.0*omega_1 + 0.1*x_pid_1 - 0.1*x_turbine_1 - 0.05*sin(0.5*delta_1 - 0.5*delta_2) + 1.0],
[                                                                                                                                       -2*omega_1 - 0.1*sin(0.5*delta_1 - 0.5*delta_2) + 2.0],
[                                                                                                                                          -157.07963267949*omega_1 + 157.07963267949*omega_2],
[                              

In [132]:
def problem(x):
    return f_n(x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7])[:,0]
    
x_0 = [0,1.0,p_load_1_val,0.0,0.0,1.0,p_load_2_val,0.0]
sol = sopt.broyden1(problem,x_0)

params_ss_sol = {'delta_1':sol[0],
                 'omega_1':sol[1],
                 'x_turbine_1':sol[2],
                 'x_pid_1':sol[3],
                 'delta_2':sol[4],
                 'omega_2':sol[5],
                 'x_turbine_2':sol[6],
                 'x_pid_2':sol[7],
                 }

params_ss_sol.update(params_ss)


In [133]:
f.subs(params_ss)

Matrix([
[                                                                                                                                           157.07963267949*omega_1 - 157.07963267949*omega_2],
[-0.321428571428571*omega_1 + 0.178571428571429*omega_2 + 0.0142857142857143*x_pid_1 + 0.0571428571428571*x_turbine_1 - 0.0785714285714286*sin(0.5*delta_1 - 0.5*delta_2) + 0.142857142857143],
[                                                                                                    -1.0*omega_1 + 0.1*x_pid_1 - 0.1*x_turbine_1 - 0.05*sin(0.5*delta_1 - 0.5*delta_2) + 1.0],
[                                                                                                                                       -2*omega_1 - 0.1*sin(0.5*delta_1 - 0.5*delta_2) + 2.0],
[                                                                                                                                          -157.07963267949*omega_1 + 157.07963267949*omega_2],
[                              

In [134]:
A = np.array(F_x.subs(params_ss_sol)).astype(np.float64)

In [135]:
w,v =np.linalg.eig(A)
print(w)

[ -2.04689947e-02+4.99062603j  -2.04689947e-02-4.99062603j
  -1.98476154e+00+0.j          -1.80968354e-01+0.2813542j
  -1.80968354e-01-0.2813542j   -1.09473350e-16+0.j
  -1.06181879e-01+0.01774231j  -1.06181879e-01-0.01774231j]


In [136]:
fig = figure(width=600, height=300)
fig.circle(w.real,w.imag )

show(fig)

In [140]:
params_ss['p_load_1']=1250/95000

f_n = sym.lambdify((u_ss), f.subs(params_ss),'numpy') # returns a numpy-ready function
x = sol
Dt = 0.002
N_steps =20000
T = np.zeros((N_steps,1))
X = np.zeros((N_steps,8))
t=0.0
for it in range(N_steps):
    t += Dt
    f_1 = f_n(x[0],x[1],x[2],x[3],x[4],x[5],x[6],x[7])[:,0]
    x_1 = x + Dt*f_1
    x = x + 0.5*Dt*(f_1 + f_n(x_1[0],x_1[1],x_1[2],x_1[3],x_1[4],x_1[5],x_1[6],x_1[7])[:,0])
    T[it,0] = t
    X[it,:] = x.T

In [141]:
fig = figure(width=600, height=300)
fig.line(T[:,0],X[:,0],  line_width=1)
fig.line(T[:,0],X[:,4],  line_width=1, color='red')
show(fig)

fig = figure(width=600, height=300)
fig.line(T[:,0],X[:,1]*60,  line_width=1)
fig.line(T[:,0],X[:,5]*60,  line_width=1, color='red')
show(fig)